In [1]:
#Random forest is used for this

In [2]:
import re
import numpy as np
import pandas as pd
from nltk.stem.porter import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [4]:
stemmer=PorterStemmer()
df = pd.read_csv(r"C:\Users\karun\Downloads\Data Set and Images used\Data Set and Images used\Personality(P).csv")

In [5]:
print(df.head(10))

   type                                              posts
0  INFJ  'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1  ENTP  'I'm finding the lack of me in these posts ver...
2  INTP  'Good one  _____   https://www.youtube.com/wat...
3  INTJ  'Dear INTP,   I enjoyed our conversation the o...
4  ENTJ  'You're fired.|||That's another silly misconce...
5  INTJ  '18/37 @.@|||Science  is not perfect. No scien...
6  INFJ  'No, I can't draw on my own nails (haha). Thos...
7  INTJ  'I tend to build up a collection of things on ...
8  INFJ  I'm not sure, that's a good question. The dist...
9  INTP  'https://www.youtube.com/watch?v=w8-egj0y8Qs||...


In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8675 entries, 0 to 8674
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    8675 non-null   object
 1   posts   8675 non-null   object
dtypes: object(2)
memory usage: 135.7+ KB
None


In [7]:
#Data Processing

In [37]:

def cleanText(text):
    text = re.sub(r'https?://(?:www\.)?(?:youtube|youtu|youtube-nocookie)\.com(?:[^\s]+)', '', text)
    text = re.sub(r'http\S+', '', text) 
    text = text.replace("|||", " ")
    text = text.lower()
    text = re.sub(r"http\S+", "", text, flags=re.MULTILINE)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    text = ' '.join(tokens)

    return text

<>:1: SyntaxWarning: invalid escape sequence '\|'
<>:1: SyntaxWarning: invalid escape sequence '\|'
C:\Users\karun\AppData\Local\Temp\ipykernel_15492\708078908.py:1: SyntaxWarning: invalid escape sequence '\|'
  """def cleanText(text):


In [28]:
df['posts'] = df['posts'].apply(cleanText)
df.head()

,type,posts,I-E,N-S,T-F,J-P,http_per_comment,music_per_comment,question_per_comment,img_per_comment,excl_per_comment,ellipsis_per_comment
0,INFJ,' intj moments sportscenter top ten plays pran...,0,0,1,0,0.0,0.02,0.04,0.0,0.06,0.30
1,ENTP,' 'm finding lack posts alarming . sex boring ...,1,0,0,1,0.0,0.00,0.08,0.0,0.00,0.38
2,INTP,"'good one _____ course , say know ; 's blessin...",0,0,0,1,0.0,0.00,0.18,0.0,0.08,0.26
3,INTJ,"'dear intp , enjoyed conversation day . esoter...",0,0,0,0,0.0,0.02,0.18,0.0,0.06,0.52
4,ENTJ,'you 're fired . 's another silly misconceptio...,1,0,0,0,0.0,0.02,0.16,0.0,0.02,0.42


In [29]:
#Dividing the 16 personalities into 4 major groups for ease
map1 = {"I": 0, "E": 1}
map2 = {"N": 0, "S": 1}
map3 = {"T": 0, "F": 1}
map4 = {"J": 0, "P": 1}
df['I-E'] = df['type'].astype(str).str[0]
df['I-E'] = df['I-E'].map(map1)
df['N-S'] = df['type'].astype(str).str[1]
df['N-S'] = df['N-S'].map(map2)
df['T-F'] = df['type'].astype(str).str[2]
df['T-F'] = df['T-F'].map(map3)
df['J-P'] = df['type'].astype(str).str[3]
df['J-P'] = df['J-P'].map(map4)


In [30]:
df['http_per_comment'] = df['posts'].apply(lambda x: x.count('http')/50)
df['music_per_comment'] = df['posts'].apply(lambda x: x.count('music')/50)
df['question_per_comment'] = df['posts'].apply(lambda x: x.count('?')/50)
df['img_per_comment'] = df['posts'].apply(lambda x: x.count('jpg')/50)
df['excl_per_comment'] = df['posts'].apply(lambda x: x.count('!')/50)
df['ellipsis_per_comment'] = df['posts'].apply(lambda x: x.count('...')/50)

In [31]:
#Building ML on 'type' column 
X = df.drop(['type','posts','I-E','N-S','T-F','J-P'], axis=1).values
y = df['type'].values


In [32]:
print(y.shape)
print(X.shape)

(8675,)
(8675, 6)


In [33]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3, random_state=5)

In [34]:
#Ml on judging/perceiving column
X5 = df.drop(['type','posts','J-P'], axis=1).values
y5 = df['J-P'].values

print("outcome shape",y5.shape)
print("input shape for machine learning data",X5.shape)

outcome shape (8675,)
input shape for machine learning data (8675, 9)


In [35]:
X5_train,X5_test,y5_train,y5_test=train_test_split(X5,y5,test_size = 0.1, random_state=5)

random_forestt = RandomForestClassifier(n_estimators=100)
random_forestt.fit(X5_train, y5_train)

Y_prediction = random_forestt.predict(X5_test)

random_forestt.score(X5_train, y5_train)
acc_random_forestt = round(random_forestt.score(X5_train, y5_train) * 100, 2)
print("Random Forest Predictions Model",round(acc_random_forestt,2,), "%")

Random Forest Predictions Model 98.22 %
